In [14]:
from tkinter import *
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.alert import Alert
import time
import pandas as pd
import csv
import re

options = webdriver.ChromeOptions()
chromedriver = r"C:\Users\master\jupyternotebook\chromedriver.exe"
driver = webdriver.Chrome( options=options)
driver.get("https://www.albamon.com/login/logout_trans.asp")

resume_list = []
match_list = []
counter = 0

WebDriverWait(driver, 1)
driver.find_element(By.ID, "DB_Name_GI").click()
driver.find_element(By.ID, "M_ID").send_keys("dj2688")
driver.find_element(By.ID, "M_PWD_default").send_keys("htdy1004")
driver.find_element(By.CLASS_NAME, "btnLogin").click()

WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "guidePop")))
driver.find_element(By.CLASS_NAME, "guideCloseBtn").click()
time.sleep(1)
driver.find_element(By.CLASS_NAME, "g1").click()
time.sleep(1)
driver.find_element(By.CLASS_NAME, "search-button").click()

ac = ActionChains(driver)
time.sleep(1)
gyeonggi_do = driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/div[1]/div/div[2]/form/div/div[1]/div[1]/div[1]/div[2]/ul/div/div[1]/li[9]/a")
ac.move_to_element(gyeonggi_do).click().perform()

time.sleep(0.5)
bucheon = driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/div[1]/div/div[2]/form/div/div[1]/div[1]/div[1]/div[3]/ul/div/div[1]/li[14]/a")
ac.move_to_element(bucheon).click().perform()

driver.find_element(By.XPATH, "//*[@id='IsOnlyHopeArea']").click() # 선택 지역 거주 인재만 누름.
time.sleep(0.5)

manufacture = driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/div[1]/div/div[2]/form/div/div[1]/div[1]/div[2]/div[2]/ul/div/div[1]/li[6]/a")
ac.move_to_element(manufacture).click().perform()

driver.find_element(By.XPATH, "//*[@id='mCSB_4_container']/li[1]/a").click() # 전체
time.sleep(0.5)

driver.find_element(By.XPATH, "//*[@id='dev_defaultSearch']/div[3]/div/div/div[1]/a").click() # 여성.
time.sleep(0.5)
driver.find_element(By.ID, "PageSize").click() # 페이지 50명씩 열람하기
driver.find_element(By.XPATH, "//*[@id='PageSize']/option[4]").click()

time.sleep(1)
card = driver.find_element(By.XPATH, "//*[@id='dev_listResult']/li[1]/a")
repeat_str = driver.find_element(By.XPATH, "//*[@id='dev_listTotalCount']").text
repeat_num = int(repeat_str)

try:
    ac.move_to_element(card).click().perform()
except:
    print("클릭이 안되어 다시 시도합니다.")
    time.sleep(1)
    ac.move_to_element(card).click().perform()

WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div[2]/div[1]/div[2]/div[1]/section/article[1]/div[2]/div[2]/ul/li[1]/span[2]")))
temp_str = ["여월동", "원종동", "성곡동", "도당동", "원미동"]
next_button = driver.find_element(By.CLASS_NAME, "btnNext") # 이력서 다음버튼
alrt = Alert(driver)
try:
    while True:
        WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div[2]/div[1]/div[2]/div[1]/section/article[1]/div[2]/div[2]/ul/li[1]/span[2]"))) # 주소 뜰떼까지 ㄱㄷ
        address = driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/div[1]/div[2]/div[1]/section/article[1]/div[2]/div[2]/ul/li[1]/span[2]").text # 주소 따와서 address에 넣음
        for i in temp_str:
            str_matching = address.find(i)
            if str_matching != -1:
                counter += 1
                match = counter, i
                print(f"{counter}. 주소가({i}) 일치합니다.")
                url = driver.current_url
                resume_list.append(url)
                match_list.append(match)
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".btnNext")))
                driver.find_element(By.CSS_SELECTOR, ".btnNext").click()
            else:
                continue
        driver.find_element(By.CLASS_NAME, "btnNext").click()
except:
    alrt.accept()
    print("알람을 껐습니다.")
    while True:
        WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div[2]/div[1]/div[2]/div[1]/section/article[1]/div[2]/div[2]/ul/li[1]/span[2]")))
        address = driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/div[1]/div[2]/div[1]/section/article[1]/div[2]/div[2]/ul/li[1]/span[2]").text
        for i in temp_str:
            str_matching = address.find(i)
            if str_matching != -1:
                counter += 1
                match = counter, i
                print(f"{counter}. 주소가({i}) 일치합니다.")
                url = driver.current_url
                resume_list.append(url)
                match_list.append(match)
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".btnNext")))
                driver.find_element(By.CSS_SELECTOR, ".btnNext").click()
            else:
                continue
        driver.find_element(By.CLASS_NAME, "btnNext").click()
finally:
    print(f"이력서 {repeat_num}건중 주소 {counter}건 매칭되었습니다.")
    f = open(r"C:\Users\master\Desktop\taehyuki\resume_list.txt", "w")
    numbers = re.findall(r'\d+', j)
    f.write("이력서 번호 리스트 입니다. \n")
    for i, j in zip(match_list, resume_list):
        numbers = re.findall(r'\d+', j)
        f.write(str(i))
        f.write(" ")
        f.write(numbers[0])
        f.write("\n")
    f.close()
    driver.quit()


1. 주소가(원미동) 일치합니다.
2. 주소가(원미동) 일치합니다.
3. 주소가(원종동) 일치합니다.
4. 주소가(원종동) 일치합니다.
5. 주소가(도당동) 일치합니다.
6. 주소가(원종동) 일치합니다.
7. 주소가(원종동) 일치합니다.
8. 주소가(원미동) 일치합니다.
9. 주소가(여월동) 일치합니다.
10. 주소가(도당동) 일치합니다.
11. 주소가(도당동) 일치합니다.
12. 주소가(도당동) 일치합니다.
13. 주소가(원종동) 일치합니다.
14. 주소가(도당동) 일치합니다.
15. 주소가(원미동) 일치합니다.
16. 주소가(원종동) 일치합니다.
17. 주소가(원미동) 일치합니다.
18. 주소가(원종동) 일치합니다.
19. 주소가(원미동) 일치합니다.
20. 주소가(원종동) 일치합니다.
21. 주소가(여월동) 일치합니다.
22. 주소가(원종동) 일치합니다.
23. 주소가(원미동) 일치합니다.
24. 주소가(도당동) 일치합니다.
25. 주소가(여월동) 일치합니다.
26. 주소가(원미동) 일치합니다.
27. 주소가(도당동) 일치합니다.
28. 주소가(원미동) 일치합니다.
29. 주소가(원종동) 일치합니다.
30. 주소가(원미동) 일치합니다.
31. 주소가(도당동) 일치합니다.
32. 주소가(여월동) 일치합니다.
33. 주소가(원종동) 일치합니다.
34. 주소가(원미동) 일치합니다.
35. 주소가(원미동) 일치합니다.
36. 주소가(원종동) 일치합니다.
37. 주소가(원종동) 일치합니다.
38. 주소가(원미동) 일치합니다.
39. 주소가(원종동) 일치합니다.
40. 주소가(원미동) 일치합니다.
41. 주소가(원종동) 일치합니다.
42. 주소가(도당동) 일치합니다.
43. 주소가(도당동) 일치합니다.
44. 주소가(도당동) 일치합니다.
45. 주소가(원종동) 일치합니다.
46. 주소가(도당동) 일치합니다.
47. 주소가(원종동) 일치합니다.
48. 주소가(원종동) 일치합니다.
49. 주소가(원종동) 일치합니다.
50. 주소가(도당동) 일치합니다.
51. 주소가(원

NoAlertPresentException: Message: no such alert
  (Session info: chrome=108.0.5359.99)
Stacktrace:
Backtrace:
	Ordinal0 [0x00C9ACD3+2075859]
	Ordinal0 [0x00C2EE61+1633889]
	Ordinal0 [0x00B2B680+571008]
	Ordinal0 [0x00B2486B+542827]
	Ordinal0 [0x00B3DD51+646481]
	Ordinal0 [0x00B77394+881556]
	Ordinal0 [0x00B3D749+644937]
	Ordinal0 [0x00B77136+880950]
	Ordinal0 [0x00B4FEFD+720637]
	Ordinal0 [0x00B50F3F+724799]
	GetHandleVerifier [0x00F4EED2+2769538]
	GetHandleVerifier [0x00F40D95+2711877]
	GetHandleVerifier [0x00D2A03A+521194]
	GetHandleVerifier [0x00D28DA0+516432]
	Ordinal0 [0x00C3682C+1665068]
	Ordinal0 [0x00C3B128+1683752]
	Ordinal0 [0x00C3B215+1683989]
	Ordinal0 [0x00C46484+1729668]
	BaseThreadInitThunk [0x75627D69+25]
	RtlInitializeExceptionChain [0x776BBB9B+107]
	RtlClearBits [0x776BBB1F+191]


In [ ]:
    print(f"이력서 {repeat_num}건중 주소 {counter}건 매칭되었습니다.")
    f = open(r"C:\Users\master\Desktop\taehyuki\resume_list.txt", "w")
    numbers = re.findall(r'\d+', j)
    f.write("이력서 번호 리스트 입니다. \n")
    for i, j in zip(match_list, resume_list):
        numbers = re.findall(r'\d+', j)
        f.write(str(i))
        f.write(" ")
        f.write(numbers[0])
        f.write("\n")
    f.close()
    driver.quit()


이력서 343건중 주소 53건 매칭되었습니다.
